# Pytorch基本操作考察

In [1]:
import torch

## 基本操作实验1
使用 𝐓𝐞𝐧𝐬𝐨𝐫 初始化一个 𝟏×𝟑 的矩阵 𝑴 和一个 𝟐×𝟏 的矩阵 𝑵，对两矩阵进行减法操作（要求实现三种不同的形式），给出结果并分析三种方式的不同（如果出现报错，分析报错的原因），同时需要指出在计算过程中发生了什么

计算过程中发生了什么？\
广播操作

使用第三种减法操作会报错，因为是原地操作，当进行广播后，计算结果和原矩阵的形状不同，所以会报错

In [2]:
M = torch.rand(1,3)
N = torch.rand(2,1)

In [3]:
M

tensor([[0.0270, 0.2390, 0.7372]])

In [4]:
N

tensor([[0.2716],
        [0.2433]])

**减法形式一**

In [5]:
M - N

tensor([[-0.2446, -0.0326,  0.4656],
        [-0.2163, -0.0042,  0.4940]])

**减法形式二**

In [7]:
result = torch.rand(2,3)
torch.sub(M,N,out=result)
result

tensor([[-0.2446, -0.0326,  0.4656],
        [-0.2163, -0.0042,  0.4940]])

**减法形式三: inplace**

In [8]:
M.sub_(N)
M

RuntimeError: output with shape [1, 3] doesn't match the broadcast shape [2, 3]

## 基本操作实验2
1) 利用 𝐓𝐞𝐧𝐬𝐨𝐫 创建两个大小分别 𝟑×𝟐 和 𝟒×𝟐 的随机数矩阵 𝑷 和 𝑸 ，要求服从均值为0，标准差0.01为的正态分布 \
2) 对第二步得到的矩阵 𝑸 进行形状变换得到 𝑸 的转置 $𝑸^𝑻$  \
3) 对上述得到的矩阵 𝑷 和矩阵 $𝑸^𝑻$ 求内积

**Answer 1):**

In [9]:
mean = 0
std = 0.01
P = torch.normal(mean,std,size=(3,2))
Q = torch.normal(mean,std,size=(4,2))

In [10]:
P

tensor([[-0.0111,  0.0123],
        [-0.0119, -0.0064],
        [-0.0007,  0.0063]])

In [11]:
Q

tensor([[ 0.0114,  0.0128],
        [ 0.0167, -0.0100],
        [-0.0115,  0.0084],
        [ 0.0016, -0.0133]])

**Answer 2):**

In [14]:
Qt = Q.t()
Qt

tensor([[ 0.0114,  0.0167, -0.0115,  0.0016],
        [ 0.0128, -0.0100,  0.0084, -0.0133]])

In [15]:
print("The shape of Q is ", Q.shape)
print("The shape of Q after transposing is ", Qt.shape)

The shape of Q is  torch.Size([4, 2])
The shape of Q after transposing is  torch.Size([2, 4])


****Answer 3):****

In [22]:
outer_product = torch.mm(P,Qt)
outer_product

tensor([[ 3.0825e-05, -3.0886e-04,  2.3096e-04, -1.8182e-04],
        [-2.1912e-04, -1.3524e-04,  8.3214e-05,  6.6380e-05],
        [ 7.2738e-05, -7.4042e-05,  6.0398e-05, -8.4600e-05]])

## 基本操作实验3
给定公式$𝑦_3=𝑦_1+𝑦_2=𝑥^2+𝑥^3$，且 $𝑥=1$。利用学习所得到的Tensor的相关知识，求$𝑦_3$对的梯度𝑥，即$𝑑𝑦_3$/$𝑑𝑥$。要求在计算过程中，在计算 $𝑥^3$ 时中断梯度的追踪，观察结果并进行原因分析

$x^3$的梯度是2而不是5的原因：\
由于 $y_2$的定义是被torch.no_grad():包裹的，所以与$y_2$有关的梯度是不会回传的，只有与$y_1$有关的梯度才会回传，即$x^2$对 $x$的梯度。\
而y2.requires_grad=False，所以不能调用 y2.backward()，会报错

In [61]:
x = torch.tensor([1.0], requires_grad=True)

In [62]:
y1 = x * x

In [63]:
with torch.no_grad():
    y2 = x * y1
y2.requires_grad

False

In [64]:
y3 = y1 + y2
y3

tensor([2.], grad_fn=<AddBackward0>)

In [66]:
y3.backward()
x.grad

tensor([2.])